In [29]:
import pymongo
from dotenv import load_dotenv
import os

load_dotenv()
mongodb_uri = os.getenv("MONGODB_URI")
client = pymongo.MongoClient(mongodb_uri)
db = client["schedule"]
classes = db["classes"]

In [30]:
ece_120 = classes.find_one({"code" : "ECE", "number" : "120"})
ece_110 = classes.find_one({"code" : "ECE", "number" : "110"})
phys_211 = classes.find_one({"code" : "PHYS", "number" : "211"})

ece_120_crns = [section["crn"] for section in ece_120["sections"]]
ece_110_crns = [section["crn"] for section in ece_110["sections"]]
phys_211_crns = [section["crn"] for section in phys_211["sections"]]

In [31]:
from datetime import datetime, timedelta

classes_to_take = [
    {
        "code" : "ECE",
        "number" : "110",
        "crn_list" : ece_110_crns
    },
    {
        "code" : "ECE",
        "number" : "120",
        "crn_list" : ece_120_crns
    },
    {
        "code" : "PHYS",
        "number" : "211",
        "crn_list" : phys_211_crns
    }
]

start_time = datetime(2021, 1, 1, 8, 0, 0) #8AM
end_time = datetime(2021, 1, 1, 17, 0, 0) #5pm
open_sections_only = True

In [32]:
for clas in classes_to_take : 
    clas_mongo_object = classes.find_one({"code" : clas["code"], "number" : clas["number"]})
    clas["sections"] = []
    for section in clas_mongo_object["sections"] :
        if section["crn"] in clas["crn_list"] :
            clas["sections"].append(section)

print(classes_to_take)

[{'code': 'ECE', 'number': '110', 'crn_list': ['36785', '36788', '36780', '36801', '36794', '55569', '36798', '55155', '36781', '55156', '36778', '36800', '36792', '36783', '36796', '62483', '55568', '36790', '36789', '62844', '59864', '59865', '59866', '59867', '59868', '59869', '59870', '59871', '59872', '59873', '59875', '59876', '59878', '59879', '59880', '62509'], 'sections': [{'crn': '36785', 'api_link': 'https://courses.illinois.edu/cisapp/explorer/schedule/2023/fall/ECE/110/36785.xml', 'section_number': 'AB0', 'status_code': 'A', 'part_of_term': '1', 'section_status_code': 'A', 'enrollment_status': 'Closed', 'section_text': 'ECE 110 cannot be taken concurrently with ECE 120.', 'start_date': datetime.datetime(2023, 8, 21, 0, 0), 'end_date': datetime.datetime(2023, 12, 6, 0, 0), 'meetings': [{'id': '0', 'type': 'Laboratory', 'type_code': 'LAB', 'start_time': datetime.datetime(1900, 1, 1, 18, 0), 'end_time': datetime.datetime(1900, 1, 1, 20, 50), 'days': None, 'room_number': '1001

In [34]:
for clas in classes_to_take :
    print(f" {clas['code']} {clas['number']} : {len(clas['sections'])} sections")

 ECE 110 : 36 sections
 ECE 120 : 17 sections
 PHYS 211 : 67 sections


In [35]:
# Remove sections that are not in preferred times

def is_preferred_time (section) :
    earliest_meeting = section["meetings"][0]['start_time']
    latest_meeting = section["meetings"][0]['end_time']
    for meeting in section["meetings"] :
        if meeting['start_time'] < earliest_meeting :
            earliest_meeting = meeting['start_time']
        if meeting['end_time'] > latest_meeting :
            latest_meeting = meeting['end_time']

    return earliest_meeting.time() > start_time.time() and latest_meeting.time() < end_time.time()


for clas in classes_to_take :
    clas["sections"] = [section for section in clas['sections'] if is_preferred_time(section)]

In [36]:
for clas in classes_to_take :
    print(f" {clas['code']} {clas['number']} : {len(clas['sections'])} sections")

 ECE 110 : 24 sections
 ECE 120 : 16 sections
 PHYS 211 : 42 sections


In [37]:
# Removes sections that are not open if open_sections_only is True

if open_sections_only :
    for clas in classes_to_take :
        clas["sections"] = [section for section in clas['sections'] if section['enrollment_status'].lower() != 'closed']


In [38]:
for clas in classes_to_take :
    print(f" {clas['code']} {clas['number']} : {len(clas['sections'])} sections")

 ECE 110 : 3 sections
 ECE 120 : 5 sections
 PHYS 211 : 8 sections
